In [114]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [115]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

In [116]:
# convert to one hot vector 
# max number will be 30 so it will be between 1 - 30 
# one_hot("amazing restaurant", 30) 

In [117]:
vocab_size = 500 

In [118]:
encoded_reviews = [one_hot(d, vocab_size) for d in reviews]

In [119]:
encoded_reviews

[[361, 136],
 [110, 285],
 [54, 211],
 [494, 109, 183],
 [198, 190, 373],
 [122, 136],
 [179, 190, 433],
 [200, 497],
 [200, 164],
 [374, 455]]

In [120]:
# we need to setup padding 
max_length = 3 
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')

In [121]:
padded_reviews

array([[361, 136,   0],
       [110, 285,   0],
       [ 54, 211,   0],
       [494, 109, 183],
       [198, 190, 373],
       [122, 136,   0],
       [179, 190, 433],
       [200, 497,   0],
       [200, 164,   0],
       [374, 455,   0]], dtype=int32)

In [122]:
model = Sequential() 
model.add(Embedding(vocab_size, 4, input_length=max_length, name="embedding")) 
model.add(Flatten()) 
model.add(Dense(1, activation="sigmoid")) 

In [123]:
X = padded_reviews 
y = sentiment 

In [124]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [125]:
model.fit(X, y, epochs=50) 

Epoch 1/50
1/1 [==============================] - 0s 346ms/step - loss: 0.6967 - accuracy: 0.6000
Epoch 2/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6953 - accuracy: 0.6000
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6939 - accuracy: 0.6000
Epoch 4/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6925 - accuracy: 0.6000
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6911 - accuracy: 0.6000
Epoch 6/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6897 - accuracy: 0.6000
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6883 - accuracy: 0.7000
Epoch 8/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6869 - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6855 - accuracy: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 4ms/step - loss: 0.6841 - accuracy: 0.8000
Epoch 11/50
1/1 [====================

In [126]:
loss, accuracy = model.evaluate(X, y)

1/1 [==============================] - 0s 106ms/step - loss: 0.6221 - accuracy: 1.0000


In [127]:
weights = model.get_layer("embedding").get_weights()[0]

In [128]:
len(weights)

500

In [129]:
weights[11]

array([ 0.01643981,  0.01034271,  0.00352401, -0.03167285], dtype=float32)

In [130]:
classifications = model.predict(X)

1/1 [==============================] - 0s 41ms/step


In [131]:
classifications 

array([[0.5234193 ],
       [0.52983975],
       [0.52930236],
       [0.5601303 ],
       [0.53691196],
       [0.47375083],
       [0.4615184 ],
       [0.45808882],
       [0.47144413],
       [0.44557813]], dtype=float32)

In [132]:
final_classifications = [(prediction > 0.5).astype(int) for prediction in classifications]

In [133]:
final_classifications

[array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([1]),
 array([0]),
 array([0]),
 array([0]),
 array([0]),
 array([0])]

In [134]:
# Now, let's prepare new data for prediction
new_reviews = ['great food',  # should be positive
               'terrible service',  # should be negative
               'awesome experience']  # should be positive

encoded_new_reviews = [one_hot(d, vocab_size) for d in new_reviews]
padded_new_reviews = pad_sequences(encoded_new_reviews, maxlen=max_length, padding='post')

In [135]:
padded_new_reviews

array([[ 19, 136,   0],
       [164, 497,   0],
       [246, 465,   0]], dtype=int32)

In [136]:
encoded_new_reviews

[[19, 136], [164, 497], [246, 465]]

In [137]:
padded_new_reviews

array([[ 19, 136,   0],
       [164, 497,   0],
       [246, 465,   0]], dtype=int32)

In [138]:
predictions = model.predict(padded_new_reviews)

1/1 [==============================] - 0s 18ms/step


In [139]:
predictions

array([[0.5140715 ],
       [0.4580804 ],
       [0.49491358]], dtype=float32)

In [140]:
binary_predictions = np.round(predictions).flatten().astype(int)

In [141]:
binary_predictions

array([1, 0, 0])

In [142]:
# Print the predictions
for review, prediction in zip(new_reviews, binary_predictions):
    print(f"Review: {review} -> Sentiment: {'Positive' if prediction == 1 else 'Negative'}")

Review: great food -> Sentiment: Positive
Review: terrible service -> Sentiment: Negative
Review: awesome experience -> Sentiment: Negative
